In [1]:
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import os
from google import genai
from google.genai import types
import torch

c:\Users\ychaf\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [3]:
device = 0 if torch.cuda.is_available() else -1

# classification model
model_name = "ychafiqui/moroccan_law_qst_classif_arabert2"
pipe = pipeline("text-classification", model=AutoModelForSequenceClassification.from_pretrained(model_name), 
                tokenizer=AutoTokenizer.from_pretrained(model_name), device=device)

# embedding model
# EMBED_MODEL = "intfloat/multilingual-e5-base"
EMBED_MODEL = "mhaseeb1604/bge-m3-law"
embedder = SentenceTransformer(EMBED_MODEL, device=device)

# generative AI model
client = genai.Client(api_key=os.environ.get("GEMINI_API_KEY"))
model = "gemini-flash-lite-latest"
generate_content_config = types.GenerateContentConfig(thinking_config=types.ThinkingConfig(thinking_budget=0))

Device set to use cuda:0


In [5]:
articles_emb = pd.read_parquet("adala.justice.gov/articles_emb.parquet")
# articles_emb["article_text_emb"] = articles_emb["article_text_emb"].apply(lambda x: eval(x))

In [6]:
def classify_qst(qst):
    return pipe(qst)[0]["label"]

In [7]:
def retrieve(query, law_category=None, k=3):
    query_embedding = embedder.encode(f"query: {query}")
    if law_category:
        filtered_articles = articles_emb[articles_emb.merged_category == law_category]
    else:
        filtered_articles = articles_emb.copy()
    similarities = cosine_similarity([query_embedding], list(filtered_articles["article_text_emb"]))[0]

    filtered_articles.loc[:, "score"] = similarities
    top_docs = filtered_articles.sort_values("score", ascending=False).head(k)
    return top_docs

In [8]:
def generate_answer(query, context):
    context = "\n\n".join(context["article_text"].tolist())
    prompt = f"""
    أجب عن السؤال باستخدام المعلومات التالية فقط.
    إذا لم تتضمن المعلومات المقدمة إجابة السؤال، فاذكر فقط أنه لا توجد معلومات كافية للإجابة.

    المعلومات:
    {context}

    السؤال:
    {query}

    الإجابة:
    """
    contents = [
        types.Content(role="user", parts=[types.Part.from_text(text=prompt)]),
    ]
    response = client.models.generate_content(
        model=model, contents=contents,
        config=generate_content_config,
    )
    return response.candidates[0].content.parts[0].text

In [20]:
question = "كيف يتم إيقاف سحب رخصة السياقة المقررة من قبل الإدارة؟"
# question = "ماهي المواد التي يجب أن توفرها الدراسة الجامعية؟"
# question = "ما هي الجهة المختصة التي يمكنني تقديم طلب إثبات الزواج أو انحلاله إليها؟"
# question = "ما هي الشروط التي يجب أن تتوفر في الموظف حتى يتم وضعه تلقائيا في وضعية التوقف المؤقت عن العمل؟"
# question = "هل يُسمح لي بتنزيل مقالات قانونية من المقالات المتاحة للجمهور على الموقع الرسمي لوزارة العدل المغربية واستخدامها ونشر ورقة بحثية؟"
law_category = classify_qst(question)
context = retrieve(question, law_category=law_category, k=5)
answer = generate_answer(question, context)
print(answer)

C:\Users\ychaf\AppData\Local\Temp\ipykernel_13680\3300014346.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_articles.loc[:, "score"] = similarities


يتوقف أثر توقيف أو سحب رخصة السياقة المقرر من قبل الإدارة وفقاً لأحكام المواد 95 و 96 و 97 أعلاه، كيفما كانت مدته، بعد أي قرار بحفظ تصدره النيابة العامة أو إ، أصبح قابال للتنفيذ، من أجل نفس الأفعال، مقرر قضائي حائز لقوة الشيء المقضي به بالبراءة أو بالإعفاء أو بعد كل ما ينهي الدعوى العمومية.

أما إذا تعلق المقرر القضائي بأداء غرامة فإن أثر التوقيف أو السحب المشار إليهما في الفقرة الأولى من هذه المادة لا يمكن أن يتوقف إلا بعد أداء الغرامة.


In [21]:
context

,category,file_name,article_number,article_text,ocr,merged_category,article_text_emb,score
11395,المادة الجنائية,مدونة السير على الطرق,216المادة,غير أنه إلغائها؛إذا كان القانون ينص على توقيف ...,False,المادة الجنائية,"[-0.026959901675581932, 0.004110859241336584, ...",0.562647
11238,المادة الجنائية,مدونة السير على الطرق,97المادة,؛بسبب حالته العقلية المركبات إما بسبب حالته ال...,False,المادة الجنائية,"[0.0018600226612761617, 0.010275539942085743, ...",0.537015
11335,المادة الجنائية,مدونة السير على الطرق,174المادة,.أعاله 173و 170و 168، عند انصرام المدة القصوى ...,False,المادة الجنائية,"[0.012597163207828999, -0.00039154384285211563...",0.534929
11239,المادة الجنائية,مدونة السير على الطرق,98المادة,.الفقرة األولى من هذه المادة ال يمكن أن يتوقف ...,False,المادة الجنائية,"[0.022639742121100426, 0.018679117783904076, -...",0.532531
11415,المادة الجنائية,مدونة السير على الطرق,229المادة,.الحامل، ويصبح االحتفاظ برخصة السياقة عندئذ غي...,False,المادة الجنائية,"[-0.024769935756921768, 0.028300857171416283, ...",0.519053


In [10]:
# question: ...?
# your question was classified to the legal category C with a confidence/probability of P%
# important terms in your query that influenced the classification: ..., ..., ...

# the 3 most similar articles to your question in the legal category C are: 
# article x (X% similarity)
# .....
# article y (Y% similarity)
# ....
# article z (Z% similarity)
# ......
# generated answer:
# ...................